## Get vectors out

In [14]:
import rasa_core
from rasa_core.agent import Agent
from bokeh.io import output_notebook
agent = Agent.load("models/dialogue_demo")
output_notebook()

INFO:tensorflow:Restoring parameters from models/dialogue_demo/policy_0_EmbeddingPolicy/tensorflow_embedding.ckpt


Loading BokehJS ...

In [15]:
policy = agent.policy_ensemble.policies[0]
interpreter = agent.interpreter
domain = agent.domain
print(policy.sparse_attention)

False


In [16]:
from rasa_core.training.structures import StoryGraph
from rasa_core.training.dsl import StoryFileReader
from rasa_core.trackers import DialogueStateTracker
from rasa_core.channels import UserMessage
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [17]:
def embedding_from_tracker(story, bot=False, predict_only=False):
    reader = StoryFileReader(domain, interpreter)
    story = story.split('\n')
    story_steps = reader.process_lines(story)
#     story_graph = StoryGraph(story_steps)
    tracker = DialogueStateTracker(
            1,
            domain.slots
    )
    tracker._reset()
    for step in story_steps:
        events = step.explicit_events(domain)
        for event in events[:-1]:
            tracker.update(event)

    if predict_only:
        return policy.predict_action_probabilities(tracker, domain)

    feed_dict=policy.tf_feed_dict_for_prediction(tracker, domain)

    dial_vec = policy.session.run(policy.dial_embed, feed_dict=feed_dict)
    bot_vec = policy.session.run(policy.bot_embed, feed_dict=feed_dict)

    no_skip_gate = policy.session.run(policy.copy_attn_debug, feed_dict=feed_dict)
    attn_embed = policy.session.run(policy.attn_embed, feed_dict=feed_dict)
    rnn_embed = policy.session.run(policy.rnn_embed, feed_dict=feed_dict)
    probs = policy.session.run(policy.alignment_history, feed_dict=feed_dict)
    if policy.all_time_masks is not None:
        time_mask = policy.session.run(policy.all_time_masks, feed_dict=feed_dict)
    else:
        time_mask = None
    if policy.topics is not None:
        topics = policy.session.run(policy.topics, feed_dict=feed_dict)
    else:
        topics = None
    cell_state = policy.session.run(policy.cell_state, feed_dict=feed_dict)
    if probs.shape[2] == 2 * probs.shape[1]:
        user_probs = probs[0, -1, :int(probs.shape[2]/2)]
        bot_probs = probs[0, -1, int(probs.shape[2]/2):]
    else:
        user_probs = probs[0, -1, :]
        bot_probs = []

    sims = policy.session.run(policy.sim_op, feed_dict=feed_dict)
    if bot is False:
        sims1 = policy.predict_action_probabilities(tracker, domain)

    del tracker
#     print(np.shape(dial_vec))

    if bot is False:
        return (dial_vec[0,-1,:],
                no_skip_gate[0,-1,:],
                attn_embed[0,-1,:],
                rnn_embed[0,-1,:],
                [user_probs, bot_probs, time_mask[0,-1,:] if time_mask is not None else time_mask],
                sims[0,-1,:],
                sims1,
                topics[0] if topics is not None else topics,
                cell_state[0]
               )
    else:
        return (dial_vec[0,-1,:],
                sims[0,-1,:],
                bot_vec[0,-1,:,:])
    
def make_prediction(story):
    dial, sim0, bot = embedding_from_tracker(story, bot=True)
    abs_vals = np.array([np.absolute(vec) for vec in bot])
    norms = np.apply_along_axis(np.linalg.norm, 1, bot)
    sims = np.dot(bot, dial)/norms
#     name = domain.action_for_index(np.argmax(sims)).name()
#     print(name)
    name = domain.action_names[np.argmax(sim0)]
#     print(name)
#     print('---')
#     ids = sim0.argsort()[::-1]
#     print(ids)
#     print(sims[ids])
    return '- ' + name, np.max(sim0)
#     print(dial.shape, bot.shape)

In [20]:
story_block = """* how_to_get_started
    - onboarding: utter_getstarted
    - onboarding: utter_first_bot_with_rasa
* affirm OR how_to_get_started{"user_type": "new"}
    - getstarted_1: action_set_onboarding
    - slot{"onboarding": true}
    - getstarted_1: utter_built_bot_before
* affirm
    - getstarted_1: utter_ask_migration
* deny
    - getstarted_1: utter_explain_stack
    - getstarted_1: utter_stack_details
    - getstarted_1: utter_explain_nlucore
* affirm OR how_to_get_started{"product":"stack"}
    - getstarted_1: utter_explain_nlu
    - getstarted_1: utter_explain_core
    - getstarted_1: utter_tryout
* how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
    - getstarted_1_success: utter_quickstart"""

# ## new to rasa at start
# * how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_getstarted_new
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore

# ## new to rasa + not new to chatbots + not migrating
# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1: utter_explain_nlu
#     - getstarted_1: utter_explain_core
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

# ## new to rasa + not new to chatbots + migrating from dialogflow

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * switch{"current_api": "dialogflow"}
#     - getstarted_1_success: utter_switch_dialogflow
#     - chitchat: utter_anything_else

# ## new to rasa + not new to chatbots + migrating from luis

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * switch{"current_api": "luis"}
#     - getstarted_1_success: utter_switch_luis
#     - chitchat: utter_anything_else

# ## new to rasa + not new to chatbots + migrating from something else

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * switch
#     - getstarted_1: action_store_unknown_product
#     - getstarted_1_success: utter_no_guide_for_switch
#     - chitchat: utter_anything_else

# ##  migrating from dialogflow

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * affirm
#     - getstarted_1: utter_ask_which_tool
# * switch{"current_api": "dialogflow"}
#     - getstarted_1_success: utter_switch_dialogflow
#     - chitchat: utter_anything_else

# ## new to rasa + not new to chatbots + migrating from luis

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * affirm
#     - getstarted_1: utter_ask_which_tool
# * switch{"current_api": "luis"}
#     - getstarted_1_success: utter_switch_luis
#     - chitchat: utter_anything_else

# ## new to rasa + not new to chatbots + migrating from something else

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * affirm
#     - getstarted_1: utter_ask_migration
# * affirm
#     - getstarted_1: utter_ask_which_tool
# * switch OR enter_data
#     - getstarted_1: action_store_unknown_product
#     - getstarted_1_success: utter_no_guide_for_switch
#     - chitchat: utter_anything_else


# ## new to rasa/bots, explain stack and try it out

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1: utter_explain_nlu
#     - getstarted_1: utter_explain_core
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

# ## new to rasa/bots, explain core and try out stack

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * how_to_get_started{"product": "core"}
#     - getstarted_1: utter_explain_core
#     - getstarted_1: utter_also_explain_nlu
# * deny
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

# ## new to rasa/bots, explain core, then nlu and try out stack

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * how_to_get_started{"product": "core"}
#     - getstarted_1: utter_explain_core
#     - getstarted_1: utter_also_explain_nlu
# * affirm
#     - getstarted_1: utter_explain_nlu
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

# ## new to rasa/bots, explain nlu and try out stack

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * how_to_get_started{"product": "nlu"}
#     - getstarted_1: utter_explain_nlu
#     - getstarted_1: utter_also_explain_core
# * deny
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

# ## new to rasa/bots, explain nlu then core and try out stack

# * how_to_get_started
#     - onboarding: utter_getstarted
#     - onboarding: utter_first_bot_with_rasa
# * affirm OR how_to_get_started{"user_type": "new"}
#     - getstarted_1: action_set_onboarding
#     - slot{"onboarding": true}
#     - getstarted_1: utter_built_bot_before
# * deny
#     - getstarted_1: utter_explain_stack
#     - getstarted_1: utter_stack_details
#     - getstarted_1: utter_explain_nlucore
# * how_to_get_started{"product": "nlu"}
#     - getstarted_1: utter_explain_nlu
#     - getstarted_1: utter_also_explain_core
# * affirm
#     - getstarted_1: utter_explain_core
#     - getstarted_1: utter_tryout
# * how_to_get_started{"product":"core"} OR affirm OR how_to_get_started{"product":"stack"}
#     - getstarted_1_success: utter_quickstart

In [21]:
print(len(story_block.split('\n')))
cool_list = []
lines = story_block.split('\n')
for idx, line in enumerate(lines):
    if idx == len(lines)-1:
        continue
    if '*' in lines[idx+1] or 'slot{' in lines[idx+1]:
#     if 'slot{' in lines[idx+1]:
        continue
    sub_story = '\n'.join(lines[:idx+1])
    (dial, not_skip, attn, rnn, probs, sims, sims1, topic, cell_state
    ) = embedding_from_tracker("#generate story\n{}".format(sub_story))
#     print(np.max(sims), np.max(sims1))

    pred, sim = make_prediction("#generate story\n{}".format(sub_story))
    cool_list.append((lines[idx+1].strip(), dial, (pred, sim, sims), not_skip, attn[-dial.shape[-1]:], rnn,
                      probs, topic, cell_state))

print(len(cool_list))
# print(cool_list[0][4].shape)
#     break
# dial = embedding_from_tracker("#generate story\n{}".format(story_block)
# )
# print(dial.shape)
# print(len(story_block.split('\n')))
# print(probs[0, 6, :])
print(dial.shape[-1])

19
12
20


In [22]:
import copy

# cool_list0 = copy.deepcopy(cool_list)
cool_list[2]

('- getstarted_1: action_set_onboarding',
 array([-0.109, -0.074,  0.154, -0.088, -0.022, -0.076, -0.221, -0.139,
        -0.16 , -0.032,  0.014,  0.167, -0.079,  0.042,  0.006, -0.259,
         0.055,  0.073, -0.063,  0.219], dtype=float32),
 ('- action_set_onboarding',
  0.8343129,
  array([ 0.252,  0.053,  0.048,  0.039,  0.055,  0.066,  0.053,  0.079,
          0.062,  0.045,  0.009,  0.044,  0.072,  0.047,  0.074,  0.041,
          0.052,  0.069,  0.069,  0.065,  0.057,  0.064,  0.037,  0.007,
          0.065,  0.073,  0.034,  0.079,  0.047,  0.037,  0.053,  0.08 ,
          0.056,  0.064,  0.071,  0.066, -0.145,  0.06 ,  0.035, -0.069,
          0.071,  0.024, -0.092, -0.069,  0.052,  0.045, -0.017,  0.062,
         -0.042,  0.019, -0.018, -0.07 ,  0.074, -0.094, -0.314,  0.007,
         -0.156, -0.403, -0.095, -0.036, -0.238, -0.169, -0.15 , -0.014,
          0.038, -0.863, -0.118,  0.029, -0.004,  0.02 ,  0.031,  0.057,
          0.371,  0.031, -0.06 , -0.066,  0.012,  0.092, -

In [23]:
# cool_list1= copy.deepcopy(cool_list)
cool_list[-1]

('- getstarted_1_success: utter_quickstart',
 array([ 0.018,  0.021,  0.173,  0.085, -0.089,  0.105, -0.084,  0.072,
         0.061, -0.07 , -0.134, -0.135, -0.023,  0.013, -0.12 ,  0.076,
         0.022,  0.017,  0.02 ,  0.046], dtype=float32),
 ('- utter_quickstart',
  0.45015877,
  array([-0.339, -0.153, -0.128, -0.14 , -0.14 , -0.129, -0.138, -0.137,
         -0.11 , -0.11 , -0.102, -0.117, -0.12 , -0.109, -0.116, -0.108,
         -0.11 , -0.13 , -0.122, -0.106, -0.147, -0.116, -0.126, -0.046,
         -0.121, -0.145, -0.095, -0.125, -0.131, -0.117, -0.119, -0.091,
         -0.116, -0.1  , -0.131, -0.139, -0.131, -0.12 , -0.11 ,  0.45 ,
          0.189,  0.298,  0.106, -0.268, -0.126, -0.114, -0.03 , -0.12 ,
         -0.081, -0.156,  0.201,  0.032,  0.154, -0.268,  0.208,  0.09 ,
         -0.24 , -0.123,  0.   , -0.194,  0.175, -0.037, -0.079,  0.357,
         -0.093,  0.126, -0.132, -0.087,  0.025,  0.   , -0.122, -0.109,
          0.124,  0.028,  0.282,  0.116,  0.401,  0.239, -0

In [24]:
from sklearn.decomposition import PCA

In [25]:
pca = PCA(n_components=2)
X = pca.fit_transform([ting[1] for ting in cool_list])

In [26]:
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt

In [27]:
embed_dim = cool_list[0][1].shape[0]

In [28]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, Arrow, OpenHead

hover = HoverTool(tooltips=[("true action","@desc"), ("prediction", "@pred"),
                            ("not_skip_gate", "@not_skip")],names=['circle'])
p = figure( tools=[hover])

for i in range(1, len(X)):
    p.add_layout(Arrow(end=OpenHead(line_color="firebrick", size=10, line_width=2),
                       x_start=X[i-1,0], y_start=X[i-1,1],
                       x_end=X[i,0], y_end=X[i,1], line_color='gray'))

source = ColumnDataSource(data=dict(
    x=X[:,0],
    y=X[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=10, name='circle')
# p.line(X[:,0], X[:,1])

X1 = pca.transform([ting[4] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X1[:,0],
    y=X1[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='green', name='circle')

X2 = pca.transform([ting[5] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X2[:,0],
    y=X2[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='red', name='circle')

x = pca.transform(np.zeros((1,embed_dim)))
p.circle(x[0,0], x[0,1], size=5, color='magenta')
p.circle(X[0,0], X[0,1], size=10, color='green')
p.circle(X[-1,0], X[-1,1], size=10, color='red')


GlyphRenderer(id='1090', ...)

In [29]:
show(p)